In [1]:
import sys
sys.path.append('../../')
sys.path.append('../')


import pandas as pd
import numpy as np

from codes.docs.analysis.nimagen import stats, visualisation, graph,genes
from codes.docs.analysis import data_preprocessing, genetic_file_preprocess

In [46]:
from joblib import Parallel, delayed
import time
import tqdm
import glob

In [30]:
def my_function(x):
    # Your function code here
    time.sleep(5)
    with open("output.txt", "a") as f:
        f.write(f"Result for x={x}: {x + 1}\n")

In [35]:
snp_file = pd.read_table('../../dataset/SNPs/asd/ASD.snp')
best_SNPs=genes.SNPsFunctionalAnalysis.get_the_best_SNPs(snp_file,threshold=0.05)

In [40]:
def create_random_genesets(job):
    random_SNPs = best_SNPs.sample(643)
    random_SNPs_annot = genes.SNPsFunctionalAnalysis(
        snps_list=random_SNPs.SNP.to_list(),
        bed_file=
        '../../dataset/genetic_dataset/target_files/batch2/euro_batch2_genotyped.bed'
    )
    random_SNPs_annot.genes_ID, random_SNPs_annot.snp_ID = random_SNPs_annot.SNPs_annotation_to_gene(
        snps_list=random_SNPs.SNP.tolist(),
        gene_build_path=
        f'../../dataset/genetic_dataset/gene_build/NCBI37.3.gene.loc',
        window_size=0)
    random_SNPs_annot.genes_ID[['Gene_ID']].to_csv(
        f'../../dataset/preprocessed_dataset/asd/GSEA/random/random_genes_all{job}.txt',
        header=False,
        index=False,
        sep=' ')

In [66]:
def generate_best_enriched_path(job):
    msigdb_dataset ='../../dataset/genetic_dataset/pathway_database/MSigDB/MSigDB_custom_entrez.gmt'
    background_gene = '../../dataset/genetic_dataset/gene_build/NCBI37_gene_loc.txt'
    gene_job = f'../../dataset/preprocessed_dataset/asd/GSEA/random/random_genes_all{job}.txt'
    res = genes.GeneSetEnrichment.ora(msigdb_dataset,background_gene,gene_job,disable_tqdm=True)
    res['run'] = job
    res = res.sort_values(by='adjP')
    output_file = '../../dataset/preprocessed_dataset/asd/GSEA/random/random_enriched_paths.csv'
    if not os.path.isfile(output_file):
        res.head(3).to_csv(output_file, header='column_names')
    else: # else it exists so append without writing the header
        res.head(3).to_csv(output_file, mode='a', header=False)
    

In [44]:
start=time.time()
tasks = [delayed(create_random_genesets)(i) for i in range(30,100)]
Parallel(n_jobs=-1)(tqdm.tqdm(tasks))
end = time.time()
print((end-start))

100%|███████████████████████████████████████████| 70/70 [01:00<00:00,  1.15it/s]


67.95244216918945


In [67]:
start=time.time()
tasks = [delayed(generate_best_enriched_path)(i) for i in range(0,3)]
Parallel(n_jobs=-1)(tqdm.tqdm(tasks))
end = time.time()
print((end-start))

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 763.99it/s]


126.59573197364807


In [43]:
start=time.time()
results=[]
for i in tqdm.tqdm(range(20,30)):
    create_random_genesets(i)
end = time.time()
print((end-start))

100%|███████████████████████████████████████████| 10/10 [00:18<00:00,  1.87s/it]

18.75246286392212
